In [4]:
!pip install transformers datasets

In [7]:
import pandas as pd
import numpy as np
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

In [8]:
# Load your cleaned and prepared dataset
df = pd.read_csv('cleaned_reviews.csv')

# Drop any potential rows that might still have missing text
df = df.dropna(subset=['review_text'])

# Display the first few rows to confirm it's loaded correctly
print(df.head())

   review_id  user_id      app_name    app_category  \
0          1  1967825     MX Player  Travel & Local   
1          2  9242600        Tinder      Navigation   
2          3  7636477       Netflix          Dating   
3          4   209031         Venmo    Productivity   
4          5  7190293  Google Drive       Education   

                                         review_text review_language  rating  \
0  Qui doloribus consequuntur. Perspiciatis tempo...              no     1.3   
1  Great app but too many ads, consider premium v...              ru     1.6   
2  The interface could be better but overall good...              es     3.6   
3  Latest update broke some features, please fix ...              vi     3.8   
4  Perfect for daily use, highly recommend to eve...              tl     3.2   

           review_date  verified_purchase     device_type  num_helpful_votes  \
0  2024-10-09 19:26:40               True  Android Tablet                 65   
1  2024-06-21 17:29:40      

In [10]:
# Convert pandas DataFrame to Hugging Face Dataset object
dataset = Dataset.from_pandas(df)

# Split the dataset into training (80%) and testing (20%) sets
train_test_split_dataset = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split_dataset['train']
test_dataset = train_test_split_dataset['test']

print("Training data shape:", train_dataset.shape)
print("Testing data shape:", test_dataset.shape)

Training data shape: (1934, 16)
Testing data shape: (484, 16)


In [11]:
# Define the model we are using
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create a function to tokenize the text
def tokenize_function(examples):
    return tokenizer(examples['review_text'], padding="max_length", truncation=True)

# Apply the tokenizer to our datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/1934 [00:00<?, ? examples/s]

Map:   0%|          | 0/484 [00:00<?, ? examples/s]

In [1]:
# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3) # 3 labels: positive, neutral, negative

# Define a function to compute metrics during evaluation
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Define the training arguments with optimizations
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=32,  # Increased batch size
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    fp16=True  # Enable mixed precision
)

# Create the Trainer instance (the rest of the code is the same)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    compute_metrics=compute_metrics
)

# Start the training
trainer.train()

NameError: name 'AutoModelForSequenceClassification' is not defined